In [1]:
# !pip install -U googlemaps

In [2]:
# !pip install censusgeocode

In [1]:
import googlemaps
from datetime import datetime

import numpy as np
import pandas as pd
import json
import redis
import re
import requests
from urllib.parse import urlencode

In [2]:
key = open('/Users/elainewei/Desktop/google-api.txt').read()

gmaps = googlemaps.Client(key=key)

In [3]:
# caching
redis_client = redis.Redis(host = 'localhost', port = 6379, db = 0)

In [5]:
def get_fips(string, update=False):
    print(string)
    if not isinstance(string, str):
        return None, None
    elif re.search(r'\(.*\)', string) and ('University' not in string) and ('College' not in string) and ('School' not in string):
        # string in parentheses are not locations
        error = 'Not a location'
        return None, error
    else:
        location_result = fetch_place(string, update) # might need a try except
        if len(location_result) == 0:
            error = 'No matched places from Google'
            return None, error
        # elif len(location_result) > 1:
        #     error = "More than one matched place from Google"
        #     return None, error
        elif not check_county(location_result):
            error = "No county matched to string from Google"
            return None, error
        else: 
            try: 
                endpoint = 'https://geo.fcc.gov/api/census/block/find'
                lat = location_result[0]['geometry']['location']['lat']
                lng = location_result[0]['geometry']['location']['lng']
                params = {'latitude': str(lat), 'longitude': str(lng), 'censusYear': '2020', 'format': 'json'}
                url_params = urlencode(params)
                r = requests.get(f"{endpoint}?{url_params}")
                return r.json()['County']['FIPS'], None
            except:
                error = "Census geocoder error"
                return None, error
            

def check_county(location_result):
    address_components = location_result[0]['address_components']
    for component in address_components:
        if 'administrative_area_level_2' in component['types']:
            return True
        # Some cities are not subjected to any counties in the place information, e.g., NYC, St. Louis, Richmond
        elif 'locality' in component['types']:
            return True
    return False


In [6]:
def fetch_place(string, update:bool = False):
    """
    takes in a string and get the json data of the place. If not found in cache then would 
    call the google map API to fetch data.
    """
    
    place_key = string
    place = redis_client.get(string)
    
    if update:
        place = None
    
    if not place:
        print('Could not find place in cache. Retrieving from Google Maps API...')
        place = gmaps.geocode(string)
        redis_client.set(place_key, json.dumps(place))
    
    else:
        print('Found place in cache, serving from redis...')
        place = json.loads(place)
        
    return place

In [19]:
# # example
# endpoint = 'https://geo.fcc.gov/api/census/block/find'
# lat = '40.6936488'
# lng = '-89.5889864'
# params = {'latitude': lat, 'longitude': lng, 'censusYear': '2020', 'format': 'json'}
# url_params = urlencode(params)
# r = requests.get(f"{endpoint}?{url_params}")
# r.json()['County']['FIPS']

'17143'

In [2]:
df = pd.read_csv('results/timeline_gpt4_new_prompt_full.csv')

In [3]:
df

,id,start_1,end_1,location_1,start_2,end_2,location_2,start_3,end_3,location_3,...,location_11,start_12,end_12,location_12,start_13,end_13,location_13,start_14,end_14,location_14
0,C001059,1952,1970,"Fresno, Fresno County, Calif.",1970,1974,"Fresno, Calif.",1978.0,1994.0,(California state assembly),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S001158,1953,1971,"Alamosa, Alamosa County, Colo.",1971,1973,(United States Army),1973.0,1976.0,(United States Army),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M001155,1967,1985,"Fort Myers, Lee County, Fla.",1985,1993,"Gainesville, Fla.",2000.0,2003.0,(Florida state house of representatives),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W000797,1966,1980,"Forest Hills, Queens County, N.Y.",1980,1984,"Dix Hills, N.Y.",1984.0,1988.0,"Gainesville, Fla.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P000591,1954,1972,"Lansing, Ingham County, Mich.",1972,1976,"Dearborn, Mich.",1976.0,1979.0,"Ann Arbor, Mich.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,B001319,1982,1996,"Enterprise, Ala.",1996,2000,"Enterprise, Ala.",2000.0,2004.0,University of Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1306,F000479,1969,1983,"Reading, Penn.",1983,1987,"York, Penn.",1987.0,1991.0,"Reading, Penn.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1307,R000618,1964,1980,"Nebraska City, Nebr.",1980,1982,"Omaha, Nebr.",1982.0,1986.0,"Chicago, Ill.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308,S001227,1975,1989,"Bridgeton, Mo.",1989,1993,"St. Louis, Mo.",1993.0,1997.0,"Kirksville, Mo.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
i = 1

while f'location_{i}' in df.columns:
    df["fips" + str(i)], df["error" + str(i)] = zip(*df["location_" + str(i)].map(get_fips))
    df.to_csv('results/gpt4_new_prompt_full_timeline_and_fips.csv', index=False)
    i += 1

print("Finished: ")
display(df)

Fresno, Fresno County, Calif.
Found place in cache, serving from redis...
Alamosa, Alamosa County, Colo.
Found place in cache, serving from redis...
Fort Myers, Lee County, Fla.
Found place in cache, serving from redis...
Forest Hills, Queens County, N.Y.
Found place in cache, serving from redis...
Lansing, Ingham County, Mich.
Found place in cache, serving from redis...
Atlanta, Fulton County, Ga.
Found place in cache, serving from redis...
Athens, Clarke County, Ga.
Found place in cache, serving from redis...
Chicago, Cook County, Ill.
Found place in cache, serving from redis...
Chicago, Cook County, Ill.
Found place in cache, serving from redis...
New Albany, Floyd County, Ind.
Found place in cache, serving from redis...
Montreal, Quebec, Canada
Found place in cache, serving from redis...
Baton Rouge, East Baton Rouge Parish, La.
Found place in cache, serving from redis...
Napoleonville, Assumption Parish, La.
Found place in cache, serving from redis...
New Orleans, Orleans Parish, 

,id,start_1,end_1,location_1,start_2,end_2,location_2,start_3,end_3,location_3,...,fips10,error10,fips11,error11,fips12,error12,fips13,error13,fips14,error14
0,C001059,1952,1970,"Fresno, Fresno County, Calif.",1970,1974,"Fresno, Calif.",1978.0,1994.0,(California state assembly),...,None,None,None,None,None,None,None,None,None,None
1,S001158,1953,1971,"Alamosa, Alamosa County, Colo.",1971,1973,(United States Army),1973.0,1976.0,(United States Army),...,None,None,None,None,None,None,None,None,None,None
2,M001155,1967,1985,"Fort Myers, Lee County, Fla.",1985,1993,"Gainesville, Fla.",2000.0,2003.0,(Florida state house of representatives),...,None,None,None,None,None,None,None,None,None,None
3,W000797,1966,1980,"Forest Hills, Queens County, N.Y.",1980,1984,"Dix Hills, N.Y.",1984.0,1988.0,"Gainesville, Fla.",...,None,None,None,None,None,None,None,None,None,None
4,P000591,1954,1972,"Lansing, Ingham County, Mich.",1972,1976,"Dearborn, Mich.",1976.0,1979.0,"Ann Arbor, Mich.",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,B001319,1982,1996,"Enterprise, Ala.",1996,2000,"Enterprise, Ala.",2000.0,2004.0,University of Alabama,...,None,None,None,None,None,None,None,None,None,None
1306,F000479,1969,1983,"Reading, Penn.",1983,1987,"York, Penn.",1987.0,1991.0,"Reading, Penn.",...,None,None,None,None,None,None,None,None,None,None
1307,R000618,1964,1980,"Nebraska City, Nebr.",1980,1982,"Omaha, Nebr.",1982.0,1986.0,"Chicago, Ill.",...,None,None,None,None,None,None,None,None,None,None
1308,S001227,1975,1989,"Bridgeton, Mo.",1989,1993,"St. Louis, Mo.",1993.0,1997.0,"Kirksville, Mo.",...,None,None,None,None,None,None,None,None,None,None


In [161]:
df = pd.read_csv('results/gpt4_new_prompt_full_timeline_and_fips.csv')

In [162]:
# manually replacing fips for locations whose names appear multiple times and cause error "More than one matched place from Google"
dict = {'Los Angeles, Calif.': '6037', 'Honolulu, Hawaii': '15003', 'Philadelphia, Philadelphia County, Pa.': '42101', 'Riverside, Calif.': '6065',  'Williamstown, Mass.': '25003', 'Saugerties, N.Y.': '36111', 'Staten Island, Richmond County, N.Y.': '36085', 'Urbana-Champaign, Ill.': '17019'}

i = 1
while f'location_{i}' in df.columns:
    df.loc[df[f'location_{i}'].isin(dict), f'fips{i}'] = df.loc[df[f'location_{i}'].isin(dict), f'location_{i}'].map(dict) # modifying only pairs with specific locations
    i += 1

In [163]:
df = df[df.columns.drop(list(df.filter(regex='location')))]
df = df[df.columns.drop(list(df.filter(regex='error')))]

In [164]:
# reshaping the dataframe
df_long = pd.wide_to_long(df, ['start_', 'end_', 'fips'], i="id", j="number")
df_long = df_long.reset_index()
df_long = df_long[df_long.columns.drop(['number'])]
df_long = df_long.rename(columns={'start_': 'start', 'end_': 'end'})

df_long = df_long.dropna(subset=['fips'])

In [165]:
# special cases

# modify special case: birth year not explicitly specified
df_long.at[599, 'start'] = 1968

# modify special case: year written as October 17, 2019
df_long.at[8073, 'start'] = 2019

# modify special case: year written as July 27 and 28, 2020
df_long.at[12058, 'start'] = 2020
df_long.at[12058, 'end'] = 2020


In [166]:
# fixing type of year

for col in ['start', 'end']:
    df_long[col] = df_long[col].replace(['present', 'Present', 'Current'], 2023)
    df_long[col] = df_long[col].replace('?', np.nan)
    df_long[col] = pd.to_numeric(df_long[col], downcast='integer')

df_long['end'] = df_long['end'].fillna(df_long['start'])


In [167]:
df_long['length'] = df_long['end'].sub(df_long['start'])
df_long['length'] = pd.to_numeric(df_long['length'], downcast='integer')
df_long

,id,start,end,fips,length
0,C001059,1952,1970.0,6019.0,18
1,S001158,1953,1971.0,8003.0,18
2,M001155,1967,1985.0,12071.0,18
3,W000797,1966,1980.0,36081.0,14
4,P000591,1954,1972.0,26065.0,18
...,...,...,...,...,...
14907,S000064,2007,2020.0,24510.0,13
15352,A000373,2022,2022.0,31055.0,0
15854,M000627,2007,2023.0,25017.0,16
16244,W000154,2009,2021.0,51510.0,12


In [168]:
max_len = max(df_long['length'])
new_columns = [f'year{i}' for i in range(1, max_len+1)]

# Create new columns filled with NaN values
df_long[new_columns] = pd.DataFrame([[pd.NaT] * (max_len)] * len(df_long), index=df_long.index)

for idx, row in df_long.iterrows():
    start_year = row['start']
    length = row['length']
    for i in range(1, length):
        df_long.at[idx, new_columns[i-1]] = start_year + i
    if length != 0:
        df_long.at[idx, new_columns[length-1]] = df_long.loc[idx]['end']

df_long = df_long.rename(columns={'start': 'year0'})
df_long = df_long.drop(['end', 'length'], axis='columns')


In [169]:
years = [f'year{i}' for i in range(max_len+1)]

# reshape the dataframe into a longer format
df_long2 = df_long.melt(id_vars=['id', 'fips'], value_vars=years, value_name='year')
df_long2 = df_long2.drop(['variable'], axis='columns')
df_long2 = df_long2.dropna(subset=['year'])
df_long2 = df_long2.sort_values(by=['id', 'year'])
df_long2 = df_long2[['id', 'year', 'fips']]
df_long2 = df_long2.drop_duplicates()

In [170]:
df_long2

,id,year,fips
195,A000014,1938,36029.0
5357,A000014,1939,36029.0
10519,A000014,1940,36029.0
15681,A000014,1941,36029.0
20843,A000014,1942,36029.0
...,...,...,...
45850,Z000018,1999,6073.0
51012,Z000018,2000,6073.0
56174,Z000018,2001,6073.0
61336,Z000018,2002,6073.0


In [171]:
df_long2.to_csv('results/gpt4_new_prompt_fips.csv', index=False)